## 自定义输出解析器

在某些情况下，您可能希望实现自定义解析器以将模型输出构造为自定义格式。

有两种方法可以实现自定义解析器：
- 在 LCEL 中使用 RunnableLambda 或 RunnableGenerator – 我们强烈建议大多数用例使用此方法
- 通过从基类之一继承进行解析——这是困难方法

这两种方法之间的差异大多是表面的，主要在于触发哪些回调（例如， on_chain_start 与 on_parser_start ）

### 可运行的 Lambda 和生成器

推荐的解析方法是使用可运行的 lambda 和可运行的生成器！

在这里，我们将进行一个简单的解析，反转模型输出的大小写。

例如，如果模型输出：“Meow”，解析器将生成“mEOW”。

In [2]:
from typing import Iterable
from langchain_core.messages import AIMessage, AIMessageChunk
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3)

def parser(aimessage: AIMessage) -> str:
    """反转大小写"""
    return aimessage.content.swapcase()

chain = chat | parser

chain.invoke("Hello!")


'hELLO! hOW CAN i ASSIST YOU TODAY?'

In [4]:
for chunk in chain.stream("tell me a story"):
    print(chunk, end="", flush=True)


oNCE UPON A TIME IN THE LUSH, VERDANT FORESTS OF eLDORIA, THERE LIVED AN OLD WISE OWL NAMED oLLIE. tHE FOREST WAS HOME TO MANY CREATURES: RABBITS THAT HOPPED THROUGH THE UNDERBRUSH, DEER THAT GRAZED ON THE SOFT GREEN GRASS, AND FOXES THAT DARTED BETWEEN TREES. bUT NONE WERE AS RESPECTED OR REVERED AS oLLIE.

oLLIE HAD SEEN COUNTLESS SEASONS PASS BY, EACH ONE BRINGING ITS OWN CHALLENGES AND JOYS. hE WAS KNOWN FOR HIS WISDOM AND KIND HEART, ALWAYS READY TO OFFER ADVICE OR A LISTENING EAR TO ANY WHO SOUGHT IT. hIS HOME WAS AN ANCIENT OAK TREE, ITS BRANCHES THICK WITH LEAVES THAT RUSTLED IN THE WIND LIKE WHISPERS OF OLD TALES.

oNE CRISP AUTUMN EVENING, AS THE SUN BEGAN TO SET AND PAINTED THE SKY IN HUES OF ORANGE AND PINK, oLLIE NOTICED SOMETHING UNUSUAL. a SMALL RABBIT, NO BIGGER THAN A HUMAN'S HAND, HOPPED INTO HIS CLEARING. tHIS WAS NOT AN ORDINARY RABBIT; IT HAD A PECULIAR MARK ON ITS FOREHEAD THAT LOOKED LIKE A TINY STAR.

tHE LITTLE RABBIT APPROACHED oLLIE WITH A LOOK OF WORRY IN IT

以上用法langchain官网不推荐，下面是推荐的使用方法

In [7]:
from langchain_core.runnables import RunnableGenerator

def stream_parser(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    """将AIMessageChunk流转换为字符串流"""
    for chunk in chunks:
        yield chunk.content.swapcase()

parser = RunnableGenerator(stream_parser)

chain = chat | parser

for chunk in chain.stream("tell me a story"):
    print(chunk, end="", flush=True)


oNCE UPON A TIME IN A LAND NOT SO FAR AWAY, THERE WAS A SMALL VILLAGE NESTLED BETWEEN ROLLING HILLS AND DENSE FORESTS. tHE VILLAGERS WERE SIMPLE FOLK WHO LIVED OFF THE LAND, GROWING CROPS AND RAISING LIVESTOCK TO SUSTAIN THEMSELVES THROUGH THE CHANGING SEASONS.

iN THIS VILLAGE, THERE LIVED AN OLD MAN NAMED eLIAN. hE WAS KNOWN FOR HIS VAST KNOWLEDGE OF HERBS AND PLANTS, WHICH HE USED TO HEAL THE SICK AND INJURED IN THE COMMUNITY. hIS SMALL HUT WAS ALWAYS FILLED WITH THE SCENT OF VARIOUS MEDICINAL PLANTS, AND PEOPLE WOULD COME FROM FAR AND WIDE SEEKING HIS HELP.

oNE CRISP AUTUMN MORNING, AS eLIAN WAS GATHERING HERBS NEAR A STREAM THAT FLOWED THROUGH THE FOREST, HE STUMBLED UPON A YOUNG GIRL HIDING BEHIND A TREE. sHE WAS NO MORE THAN TEN YEARS OLD, HER CLOTHES TATTERED AND DIRTY, AND SHE LOOKED FRIGHTENED AND LOST.

eLIAN APPROACHED HER GENTLY, SPEAKING IN SOFT TONES TO CALM HER DOWN. tHE GIRL EVENTUALLY OPENED UP, REVEALING THAT SHE HAD BEEN SEPARATED FROM HER FAMILY DURING A RAID BY B